In [1]:
import os

import ioh
import numpy as np
import modcma.c_maes as modcma

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import importlib.util

# The path to the lumapi.py file
file_path = 'C:\\Program Files\\Lumerical\\v241\\api\\python\\lumapi.py'
spec_win = importlib.util.spec_from_file_location('lumapi', file_path)
lumapi = importlib.util.module_from_spec(spec_win)
spec_win.loader.exec_module(lumapi)

import scipy.io



In [ ]:

def immersed_grating_fdtd(params, polarization, wavelength_i, polarization_angle):
    """
    runs the fdtd simulation for the immersion grating with parameters 
    params: 5 widths in x, 5 widths in z, and height H in y  [in m]
    polarization, wavelength_i and polarization_angle defines the input plane-wave source. 
    """
    fdtd = lumapi.FDTD(hide=True)

    fdtd.switchtolayout
    fdtd.deleteall

    period = 2.07e-6 #period P of the grating
    incidence_angle = 62.6 #incidence angle of the source

    #addstructuregroup();
    # Add and configure the "alGrating" structure group
    fdtd.addstructuregroup()
    fdtd.set("name", "alGrating")
    fdtd.set("enabled", 1)

    fdtd.addstructuregroup()
    fdtd.set("name", "coatingSlab")
    fdtd.set("enabled", 1)

    fdtd.addstructuregroup()
    fdtd.set("name", "etchSlab")
    fdtd.set("enabled", 1)

    fdtd.addstructuregroup()
    fdtd.set("name", "immersionSlab")
    fdtd.set("enabled", 1)

    widths_x, widths_z, H = params[:5], params[5:10], params[-1] 

    ################################### Al gratings ###################################
  
    #widths_x, widths_z, H = params[:5], params[5:10], params[-1] 

    material = "Al (Aluminium) - Palik"
    
    centers_z = np.array([0.5, 1.5, 2.5, 3.5, 4.5])*(period/5) #centers for the 5 subcell in the lattice with period P
    centers_x = np.array([0.5, 1.5, 2.5, 3.5, 4.5])*(period/5) # same as above in z dimension

    # Loop over widths in x-dimension
    for xi in range(len(widths_x)):
        center_x = centers_x[xi]
        width_x = widths_x[xi]
        
        # Loop over widths in z-dimension
        for zi in range(len(widths_z)):
            center_z = centers_z[zi]
            width_z = widths_z[xi]

            #print(f"{width_x} - {width_z}")
            
            # Create a rectangular pillar
            fdtd.addrect()
            fdtd.set('x', center_x)  # Center position in x
            fdtd.set('x span', width_x)  # Width in x-dimension
            fdtd.set('y', -H / 2)  # Base position in y
            fdtd.set('y span', H)  # Height in y-dimension
            fdtd.set('z', center_z)  # Center position in z
            fdtd.set('z span', width_z)  # Width in z-dimension
            
            fdtd.set('material', material)  # Set the material
            fdtd.set('name', f'grating_x{xi+1}_z{zi+1}')  # Unique name for each rectangle
            fdtd.addtogroup("alGrating")  # Add to the group


    ################################### Coating slab ###################################
    coating_material = "Al (Aluminium) - Palik"
    offset = period * 0.2  # 20% of the period

    ymin = 0
    ymax = 0.4e-6  # 0.4 microns coating width

    slabs_width = period + offset
    slabs_center = period / 2

    # Add a rectangle and set properties
    fdtd.addrect()
    fdtd.set("x", slabs_center)  # Center position in x
    fdtd.set("x span", slabs_width)  # Width in x-dimension
    fdtd.set("y min", ymin)  # Base position in y
    fdtd.set("y max", ymax)  # Height in y-dimension
    fdtd.set("z", slabs_center)  # Center position in z
    fdtd.set("z span", slabs_width)  # Width in z-dimension
    fdtd.set("material", coating_material)  # Set the material
    fdtd.set("name", "coatingSlab")
    fdtd.addtogroup("coatingSlab")  # Add to the group


    ################################### Etch slab ###################################
    coating_material = "etch"  # Assuming "etch" is a placeholder for the actual material name
    offset = period * 0.2  # 20% of the period

    ymin = 0.4e-6  # Start of etch slab in y-direction
    ymax = 0.6e-6  # End of etch slab in y-direction

    slabs_width = period + offset  # Width of the slab including offset
    slabs_center = period / 2  # Center position of the slab

    # Add a rectangle for the etch slab
    fdtd.addrect()
    fdtd.set("x", slabs_center)  # Center position in x
    fdtd.set("x span", slabs_width)  # Width in x-dimension
    fdtd.set("y min", ymin)  # Base position in y
    fdtd.set("y max", ymax)  # Height in y-dimension
    fdtd.set("z", slabs_center)  # Center position in z
    fdtd.set("z span", slabs_width)  # Width in z-dimension
    fdtd.set("material", coating_material)  # Set the material
    fdtd.set("name", "etchSlab")
    fdtd.addtogroup("etchSlab")  # Add to the group

    ################################### Immersion slab ###################################
    immersion_material = "Si (Silicon) - Palik"  # Silicon material from Palik's database
    offset = period * 0.2  # 20% of the period

    ymin = -0.8e-6  # Start of immersion slab in y-direction
    ymax = 0.0  # End of immersion slab in y-direction

    slabs_width = period + offset  # Width of the slab including offset
    slabs_center = period / 2  # Center position of the slab

    # Add a rectangle for the immersion slab
    fdtd.addrect()
    fdtd.set("x", slabs_center)  # Center position in x
    fdtd.set("x span", slabs_width)  # Width in x-dimension
    fdtd.set("y min", ymin)  # Base position in y
    fdtd.set("y max", ymax)  # Height in y-dimension
    fdtd.set("z", slabs_center)  # Center position in z
    fdtd.set("z span", slabs_width)  # Width in z-dimension

    # Mesh settings
    fdtd.set("override mesh order from material database", 1)
    fdtd.set("mesh order", 3)  # Set the mesh order explicitly

    # Set the material and other properties
    fdtd.set("material", immersion_material)  # Set the material
    fdtd.set("name", "immersionSlab")
    fdtd.addtogroup("immersionSlab")  # Add to the group


    ################################### index monitor ###################################
    offset = period * 0.05  # 5% of the period
    ymin = -0.75e-6  # Start of index monitor region in y-direction
    ymax = 0.5e-6  # End of index monitor region in y-direction

    slabs_width = period + offset  # Width of the index monitor including offset
    slabs_center = period / 2  # Center position of the index monitor

    # Add an index monitor
    fdtd.addindex()

    # Set position for the index monitor
    fdtd.set("x", slabs_center)  # Center position in x
    fdtd.set("x span", slabs_width)  # Width in x-dimension
    fdtd.set("y min", ymin)  # Base position in y
    fdtd.set("y max", ymax)  # Height in y-dimension
    fdtd.set("z", slabs_center / 5)  # Center position in z
    
    fdtd.set("name", "index_monitor")
    fdtd.set("monitor type", 3)  # 2D z-normal monitor type



    ################################### FDTD ###################################
    ymin = -0.75e-6; ymax = 0.5e-6; #immersion slab from y=0 to y=-0.8 um

    slabs_width = period
    slabs_center = slabs_width/2

    fdtd.addfdtd()
    fdtd.set("dimension", '3D')  # 3D
    fdtd.set('x',slabs_center)  # Center position in x
    fdtd.set('x span', slabs_width)  # Width in x-dimension

    fdtd.set('y min', ymin)  # Base position in y
    fdtd.set('y max',ymax)  # Height in y-dimension

    fdtd.set('z', slabs_center/5);  # Center position in z
    fdtd.set('z span', slabs_width/5)  # Width in z-dimension

    fdtd.set('x min bc', 'Bloch')
    fdtd.set('z min bc', 'Bloch')

    fdtd.set("mesh accuracy", 1)  # Set the mesh order explicitly


    ################################### Source ###################################
    fdtd.addplane()
    fdtd.set('injection axis', 'y') # source coming in XZ plane
    fdtd.set('x', period / 2) 
    y_source = ymin * 0.90 # 
    fdtd.set('y', y_source)
    fdtd.set('z', period / 5 / 2)
    fdtd.set('angle theta', incidence_angle)
    fdtd.set('angle phi', 90)
    if polarization == "U":
        fdtd.set('polarization angle', polarization_angle)
    else:
        fdtd.set("polarization definition", polarization)
    fdtd.set('plane wave type', 'Bloch/periodic')
    fdtd.set('wavelength start', wavelength_i)
    fdtd.set('wavelength stop', wavelength_i)

    ################################### reflection monitor ###################################
    offset = period * 0.5  # Additional offset
    slabs_width = period + offset
    slabs_center = period / 2

    fdtd.addpower()
    fdtd.set("name", "reflection_grating")
    fdtd.set("monitor type", "3D")
    fdtd.set('x', slabs_center)
    fdtd.set('x span', slabs_width)
    y_monitor = ymin * 0.95
    fdtd.set('y', y_monitor)
    fdtd.set('y min', y_monitor)
    fdtd.set('y max', y_monitor)
    fdtd.set('z', slabs_center / 5)
    fdtd.set('z span', slabs_width / 5)

   
    # Run the simulation
    fdtd.save('2dDesign2.fsp')
    index_profile = fdtd.getresult("index_monitor","index")
        
    # Run the simulation
    run_switch = 1
    if run_switch == 1:
        fdtd.run()
        Tr = fdtd.transmission("reflection_grating", 1)
        
        mname = "reflection_grating"  # monitor name
        G = fdtd.grating(mname) 
        n_orders = fdtd.gratingn(mname) 
        m_orders = fdtd.gratingm(mname)  
        u1_vectors = fdtd.gratingu1(mname)
        u2_vectors = fdtd.gratingu1(mname)
        return n_orders, m_orders, u1_vectors, u2_vectors, np.array(-Tr * G), index_profile
    else:
        return np.zeros([1,2]),np.zeros([1,2]), np.zeros([1,2]),  np.zeros([1,2]), np.zeros([1,2]), index_profile



In [20]:

def problem(params_norm: np.ndarray) -> float:
    """
    normalized immersed_grating_fdtd
    function used as problem for CMA; used "immersed_grating_fdtd"
    that takes in parameters in meters [m].
    This function takes in normalized parameters and converts
    them back to [m] to be used with "immersed_grating_fdtd".
    It returns the FOM, in this case avg. efficiency over the SWIR-3 band.
    """

    # Should return eff
    # Should normalize between -1, 1 [x0]

    widths_x_norm, widths_z_norm, H_norm = params_norm[:5], params_norm[5:10], params_norm[-1] 

    period = 2.07e-6
    widths_x = (widths_x_norm + 1) * (period/5.1 / 2)
    widths_z = (widths_z_norm + 1) * (period/5.1 / 2)
    H = (H_norm + 1) * (0.6e-6 - 0.2e-6) / 2 + 0.2e-6

    params = np.concatenate([widths_x, widths_z, [H]])

    wavelength_1, wavelength_2, wavelength_3 = 2.304e-6, 2.345e-6, 2.386e-6

    n_orders1, _, _, _, eff1, index_profile1 = immersed_grating_fdtd(params, 'U', wavelength_1, 45)
    n_orders2, _, _, _, eff2, index_profile2 = immersed_grating_fdtd(params, 'U', wavelength_2, 45)
    n_orders3, _, _, _, eff3, index_profile3 = immersed_grating_fdtd(params, 'U', wavelength_3, 45)

    eff_fom = (eff1[0] + eff2[0] + eff3[0])/3

    if eff_fom <= 0.6:
        print(f"Params = {params}, efficiency = {eff_fom[0]:0.3f}")

    return -1*eff_fom[0]


def run_cma(
    f: ioh.ProblemType,
    n_evals: int = 1,
) -> None:
    """
    # x0 = Normalized variants of any of the Top-k direction
    sigma0 = 0.1 # Normalize this also (search in radius around a point)
    thres when the algorithm has been
    """
    settings = modcma.parameters.Settings(
        dim=f.meta_data.n_variables,
        # Zero initialization
        x0=np.zeros(11),
        # σ = 0.3(b − a)
        sigma0=0.3 * (f.bounds.ub[0] - f.bounds.lb[0]),
        budget=n_evals,
        target=-1.0,
        lb=f.bounds.lb,
        ub=f.bounds.ub,
        verbose=True,
    )
    settings.modules.active = True
    settings.modules.mirrored = modcma.options.MIRRORED
    settings.modules.sampler = modcma.options.SOBOL
    settings.modules.restart_strategy = modcma.options.RESTART
    settings.modules.repelling_restart = True
    settings.modules.bound_correction = modcma.options.SATURATE

    cma = modcma.ModularCMAES(settings)
    print(settings)
    while cma.step(f):
        """Log stats for every generation"""
        print(cma.p.mutation.sigma)
    print(f.state)


def load_data() -> [np.array, np.array]:
    files = [np.load(f) for f in os.listdir() if f.endswith("npz")]
    X, y = np.empty((11, 0)), np.empty((0,))
    for file in files:
        eff, parms = sorted(list(file.keys()))
        if "eff" not in eff:
            eff, parms = parms, eff
        y = np.r_[y, file[eff]]
        X = np.c_[X, file[parms].T]
    return X, y


class ClosestProblem:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.n_points = 5

    @staticmethod
    def from_norm(x):
        widths_x_norm, widths_z_norm, H_norm = x[:5], x[5:10], x[-1]

        period = 2.07e-6
        widths_x = (widths_x_norm + 1) * (period / 5.1 / 2)
        widths_z = (widths_z_norm + 1) * (period / 5.1 / 2)
        H = (H_norm + 1) * (0.6e-6 - 0.2e-6) / 2 + 0.2e-6
        return np.concatenate([widths_x, widths_z, [H]])

    def __call__(self, x):
        assert len(x) == 11
        x_raw = self.from_norm(x)
        distances = np.sqrt(np.sum((self.X - x_raw.reshape(-1, 1)) ** 2, axis=0))
        min_dist_idx = np.argsort(distances)[: self.n_points]
        weights = 1 - distances[min_dist_idx] / distances[min_dist_idx].sum()
        weights /= weights.sum()
        return -1 * self.y[min_dist_idx] @ weights


def run_with_surrogate():
    X, y = load_data()
    p = ClosestProblem(X, y)
    ioh_problem = ioh.wrap_problem(
        p, name="metagrating_surrogate", dimension=11, lb=-1, ub=1
    )
    run_cma(ioh_problem)



In [23]:

def main():
    """Run (mu/mu_w, lambda)-CMA-ES"""

    modcma.utils.set_seed(3)
    n_reps = 3
    ioh_problem = ioh.wrap_problem(
        problem, name="metagrating", dimension=11, lb=-1, ub=1
    )
    logger = ioh.logger.Analyzer(
        triggers=[ioh.logger.trigger.ALWAYS],
        folder_name="data_loc_CMAES",
        algorithm_name="CMA-ES",
        store_positions=True,
    )
    ioh_problem.attach_logger(logger)
    for _ in range(n_reps):
        run_cma(ioh_problem, 441)
        ioh_problem.reset()


if __name__ == "__main__":
    main()


<Settings dim: 11 modules: <Modules elitist: false active: true orthogonal: false sequential_selection: false threshold_convergence: false sample_sigma: false weights: DEFAULT sampler: SOBOL mirrored: MIRRORED ssa: CSA bound_correction: SATURATE restart_strategy: RESTART matrix_adaptation: COVARIANCE> target: -1 max_generations: None budget: 441 sigma0: 0.6 lambda0: 11 mu0: 5 x0: 0
0
0
0
0
0
0
0
0
0
0 lb: -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ub: 1 1 1 1 1 1 1 1 1 1 1 cs: None cc: None cmu: None c1: None verbose: true>
Params = [1.65284559e-07 1.92905051e-07 2.89183970e-07 2.92221410e-07
 6.01306622e-08 2.90686807e-07 2.43004014e-07 1.49670738e-07
 3.02638371e-07 2.34318731e-07 3.97473143e-07], efficiency = 0.455
Params = [2.40597793e-07 2.12977302e-07 1.16698382e-07 1.13660943e-07
 3.45751691e-07 1.15195546e-07 1.62878339e-07 2.56211615e-07
 1.03243982e-07 1.71563622e-07 4.02526857e-07], efficiency = 0.283
Params = [2.25477639e-07 2.96863051e-07 1.46355301e-07 2.19988755e-07
 3.30892320e-0

KeyboardInterrupt: 

440